In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

import matplotlib.pyplot as plt
import seaborn as sns

from typing import List
from tqdm import tqdm
import os
import re

In [2]:
train = json.load(open('./input/cooking_train.json', 'r'))
test = json.load(open('./input/cooking_test.json', 'r'))

In [3]:
len(train)

30000

In [4]:
len(test)

9774

In [5]:
train[0]

{'cuisine': 'chinese',
 'id': 29565,
 'ingredients': ['romaine lettuce',
  'sliced almonds',
  'vegetable oil',
  'scallions',
  'soy sauce',
  'cooked chicken',
  'napa cabbage',
  'chopped cilantro fresh',
  'sugar',
  'sesame seeds',
  'wonton wrappers',
  'fresh lemon juice',
  'white vinegar',
  'black pepper',
  'sesame oil',
  'salt',
  'snow peas']}

In [6]:
all_data = train + test

## Count unique ingredients

In [7]:
train_meta = pd.DataFrame(index=[r['id'] for r in train], data={
    'ingred_len': [len(r['ingredients']) for r in train],
    'cuisine': [r['cuisine'] for r in train],
    'train': 1
})

test_meta = pd.DataFrame(index=[r['id'] for r in test], data={
    'ingred_len': [len(r['ingredients']) for r in test],
    'train': 0
})

In [8]:
train_meta.head()

,ingred_len,cuisine,train
29565,17,chinese,1
15528,8,italian,1
38015,15,cajun_creole,1
20511,19,italian,1
44111,14,chinese,1


In [9]:
meta = pd.concat([train_meta, test_meta], sort=True)
len(meta)

39774

# Analyzing ingredients

In [10]:
%%time
ingred_cuisine_dfs = [
    pd.DataFrame({
        'ingredient': recipe['ingredients'],
        recipe['cuisine']: 1
    }) for recipe in tqdm(train)
]

100%|██████████| 30000/30000 [00:16<00:00, 1803.94it/s]

CPU times: user 16.5 s, sys: 736 ms, total: 17.2 s
Wall time: 16.6 s


In [11]:
%%time
ingredient_cuisines = pd.concat(ingred_cuisine_dfs, sort=False).fillna(0).groupby('ingredient').sum()

CPU times: user 33.7 s, sys: 1.48 s, total: 35.2 s
Wall time: 30.7 s


In [12]:
ingredient_cuisines.head()

,chinese,italian,cajun_creole,southern_us,spanish,british,mexican,korean,indian,thai,irish,filipino,greek,jamaican,vietnamese,french,moroccan,japanese,russian,brazilian
ingredient,,,,,,,,,,,,,,,,,,,,
( oz.) tomato sauce,0.0,6.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
( oz.) tomato paste,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(10 oz.) frozen chopped spinach,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(14.5 oz.) diced tomatoes,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(15 oz.) refried beans,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's analyze what actually is in the labels - some signs such as brackets can be easily removed to eliminate noise from the data. If multiple labels contain numerical values, we can easily separate them into meaningful features.

In [13]:
ingredient_cuisines['count_notalpha'] = [len(re.findall('[^a-zA-Z\s]', i)) for i in ingredient_cuisines.index]
ingredient_cuisines['count_numbers'] = [len(re.findall('\d+', i)) for i in ingredient_cuisines.index]

In [14]:
ingredient_cuisines['count_notalpha'].agg(['min', 'max', 'mean'])

min     0.000000
max     7.000000
mean    0.085266
Name: count_notalpha, dtype: float64

In [15]:
ingredient_cuisines['count_numbers'].agg(['min', 'max', 'mean'])

min     0.000000
max     4.000000
mean    0.006239
Name: count_numbers, dtype: float64

In [16]:
len(ingredient_cuisines[ingredient_cuisines['count_numbers'] > 0])

35

##### This does not seem to be imporatnt, let's eliminate as much words as possible instead

In [18]:
len(ingredient_cuisines.index.unique())

6251

In [20]:
all_words = " ".join(ingredient_cuisines.index.unique()).split(" ")
unique_words, counts = np.unique(all_words, return_counts=True)
word_counts = pd.DataFrame({
    'word': unique_words,
    'count': counts
}).sort_values(by='count')
word_counts.head()

,word,count
3386,épices,1
1855,hothouse,1
1852,hoop,1
1851,hong,1
1849,honeycomb,1


In [24]:
len(word_counts)

3387

In [34]:
ignored_words = word_counts[word_counts['count'] == 1]
len(ignored_words)

1845

Obviously, words that occur only once have should have no meaning for our model.
This way, we can greatly reduce the feature space.

In [39]:
alnum_meaningful = [
    word for word in word_counts['word'] 
    if word.isalnum() and not word in set(ignored_words['word'])
]
len(alnum_meaningful)

1483

# Generating vectors

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Set

In [42]:
%%time
all_recipes = train + test
print(len(all_recipes))

39774
CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 6.58 ms


In [65]:
def preprocess_ingredients_whitelist(
        recipes: List[dict], 
        whitelist: Set[str] = set(alnum_meaningful)
) -> str:
    process_ingredient = lambda ingredient: " ".join([
        word for word in ingredient.split(" ") if word in whitelist
    ])
    process_recipe = lambda recipe: " ".join([
        process_ingredient(ingredient) for ingredient in recipe['ingredients']
    ])
    return [re.sub('\s+', " ", process_recipe(recipe)) for recipe in recipes]

In [62]:
def preprocess_ingredients(recipe_list: List[str]) -> str:
    keep_text_ws = lambda ingredient: "".join(re.findall('[a-zA-Z\s]', ingredient))
    strip_ingredient = lambda ingredient: "".join([word.lower() for word in keep_text_ws(ingredient).split(" ")])
    return ", ".join([strip_ingredient(ingredient) for ingredient in recipe_list])

In [68]:
preprocess_ingredients(all_recipes[0]['ingredients'])

'romainelettuce, slicedalmonds, vegetableoil, scallions, soysauce, cookedchicken, napacabbage, choppedcilantrofresh, sugar, sesameseeds, wontonwrappers, freshlemonjuice, whitevinegar, blackpepper, sesameoil, salt, snowpeas'

In [69]:
preprocess_ingredients_whitelist(all_recipes[:1])

['romaine lettuce sliced almonds vegetable oil soy sauce cooked chicken napa cabbage chopped cilantro fresh sugar sesame seeds wonton wrappers fresh lemon juice white vinegar black pepper sesame oil salt snow peas']

In [71]:
%%time
vectorizer = TfidfVectorizer()
# all_ingredients = [preprocess_ingredients(r['ingredients']) for r in all_recipes]
all_ingredients = preprocess_ingredients_whitelist(all_recipes)
all_vectors = vectorizer.fit_transform(all_ingredients)
print(type(all_vectors))
assert(len(all_recipes) == all_vectors.shape[0])
print(all_vectors.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(39774, 1337)
CPU times: user 1.55 s, sys: 20 ms, total: 1.57 s
Wall time: 1.57 s


# Generating features

In [72]:
meta_feature_columns = ['ingred_len']
meta.head()

,cuisine,ingred_len,train
29565,chinese,17,1
15528,italian,8,1
38015,cajun_creole,15,1
20511,italian,19,1
44111,chinese,14,1


In [73]:
features = meta[meta_feature_columns].values
features.shape

(39774, 1)

# Assembling model input

In [74]:
import scipy as sp
from sklearn.preprocessing import LabelEncoder

In [75]:
assert(all_vectors.shape[0] == features.shape[0])
data = sp.sparse.hstack([all_vectors, sp.sparse.csr_matrix(features)], format='csr')
type(data)

scipy.sparse.csr.csr_matrix

In [76]:
cousine_names = [r['cuisine'] for r in train]
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(cousine_names)

In [77]:
train_data = data[:len(labels)]
test_data = data[len(labels):]

# Model training and cross-validation

In [83]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

In [85]:
%%time
splitter = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
results = []
for fold, (train_idx_, eval_idx_) in enumerate(splitter.split(train_data, labels)):
    train_X_, train_y_ = train_data[train_idx_], labels[train_idx_]
    eval_X_, eval_y_ = train_data[eval_idx_], labels[eval_idx_]
    model_params = {
        'objective': 'multiclass',
        'n_estimators': 512,
        'num_class': len(np.unique(labels)),
        'n_jobs': 12,
        'random_state': 42,
        'silent': True,
    }
    fit_params = {
        'eval_set': (eval_X_, eval_y_),
        'eval_metric': 'multi_error',
        'early_stopping_rounds': 64,
        'verbose': 16,
    }
    model = LGBMClassifier(**model_params)
    model.fit(train_X_, train_y_, **fit_params)
    score = model.score(eval_X_, eval_y_)
    print(f"Fold {fold}, val_accuracy={score}")
    results.append({
        'score': score,
        'model': model
    })

Training until validation scores don't improve for 64 rounds.
[16]	valid_0's multi_logloss: 1.28911	valid_0's multi_error: 0.303131
[32]	valid_0's multi_logloss: 1.03479	valid_0's multi_error: 0.268754
[48]	valid_0's multi_logloss: 0.918036	valid_0's multi_error: 0.248501
[64]	valid_0's multi_logloss: 0.857375	valid_0's multi_error: 0.242372
[80]	valid_0's multi_logloss: 0.825538	valid_0's multi_error: 0.236109
[96]	valid_0's multi_logloss: 0.807313	valid_0's multi_error: 0.232378
[112]	valid_0's multi_logloss: 0.797314	valid_0's multi_error: 0.231712
[128]	valid_0's multi_logloss: 0.791871	valid_0's multi_error: 0.228514
[144]	valid_0's multi_logloss: 0.789347	valid_0's multi_error: 0.226782
[160]	valid_0's multi_logloss: 0.789473	valid_0's multi_error: 0.226649
[176]	valid_0's multi_logloss: 0.791395	valid_0's multi_error: 0.227182
[192]	valid_0's multi_logloss: 0.794216	valid_0's multi_error: 0.225183
[208]	valid_0's multi_logloss: 0.79752	valid_0's multi_error: 0.225716
Early stopp

# Submission generation
For a start, we will just perform simple voting from out-of-fold predictions

In [86]:
sample_subm = pd.read_csv('./input/sample_submission.csv')
print(sample_subm.shape)
sample_subm.head()

(9774, 2)


,Id,cuisine
0,24888,italian
1,43564,italian
2,21898,italian
3,6991,italian
4,37700,italian


In [87]:
result_ids = [r['id'] for r in test]
print(f"Using label encored: {label_encoder}")
print(f"Using result ids: {result_ids[:7]}...")

def generate_predictions(model_data) -> pd.DataFrame:
    model = model_data['model']
    preds = model.predict(test_data, num_iteration=model.best_iteration_)
    pred_names = label_encoder.inverse_transform(preds)
    return pd.DataFrame({
        'id': result_ids,
        'cuisine': pred_names
    })

Using label encored: LabelEncoder()
Using result ids: [24888, 43564, 21898, 6991, 37700, 43546, 20544]...


In [88]:
%%time
subm_dfs = [generate_predictions(model_data) for model_data in results]

CPU times: user 16.4 s, sys: 144 ms, total: 16.5 s
Wall time: 1.39 s


In [89]:
subm = pd.concat(subm_dfs)
print(subm.shape)
subm.head()

(39096, 2)


,id,cuisine
0,24888,italian
1,43564,spanish
2,21898,southern_us
3,6991,moroccan
4,37700,spanish


In [90]:
%%time
_sf = subm.groupby('id').cuisine.apply(lambda arr: sp.stats.mode(arr).mode[0])
subm_final = pd.DataFrame({
    'Id': _sf.index,
    'cuisine': _sf.values
})

CPU times: user 2.43 s, sys: 96 ms, total: 2.53 s
Wall time: 2.27 s


/home/kk385830/miniconda3/envs/kaggle-cooking/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [91]:
subm_final.head()

,Id,cuisine
0,16,indian
1,22,mexican
2,24,southern_us
3,32,chinese
4,48,indian


In [92]:
# sanity checks
assert(subm_final.notna().all().all())
assert(sorted(sample_subm['Id'].unique()) == sorted(subm_final['Id'].unique()))
assert(sample_subm.shape == subm_final.shape)

In [94]:
scores = [model_data['score'] for model_data in results]
mean_cv_score = np.mean(scores)
std_cv_score = np.std(scores)
model_name = 'LGBM-less-features'
subm_filename = f'{model_name}-cvmean={mean_cv_score:.4f}-cvstd={std_cv_score:.4f}.csv'
subm_path = os.path.join('./submissions/', subm_filename)
subm_path

'./submissions/LGBM-less-features-cvmean=0.7637-cvstd=0.0081.csv'

In [95]:
subm_final.to_csv(subm_path, index=False)

In [96]:
!kaggle competitions submit -f {subm_path} -m "Less features" ml1819-whats-cooking

100%|████████████████████████████████████████| 136k/136k [00:02<00:00, 50.8kB/s]
Successfully submitted to ML1819 - What's Cooking?

# Possible improvements

- For a baseline, we have only vectorized words, so multi-word ingredients are treated the same as single-word ones - 
  it may be beneficial to separate the ingredient and its modifiers
- TfIdf does not take position on the list into account - need to try other vectorization techniques
- Testing other models: NNs in particular might work well on such dataset - if well-made we can use them 
  to take order and comma-separation of ingredients vs their modifiers into account